# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../Weatherpy/weather_data.csv")
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ostrovnoy,68.0531,39.5131,39.40,93,100,10.02,RU,1655781480
1,Mahébourg,-20.4081,57.7000,68.29,56,20,5.75,MU,1655781480
2,Qaanaaq,77.4840,-69.3632,36.54,85,96,3.65,GL,1655781481
3,Zhuhai,22.2769,113.5678,87.85,89,40,10.36,CN,1655781482
4,Cape Town,-33.9258,18.4232,59.38,81,0,11.01,ZA,1655781401
...,...,...,...,...,...,...,...,...,...
543,Yabrūd,33.9674,36.6574,60.91,38,0,1.48,SY,1655781880
544,Paraíso,9.8383,-83.8656,66.88,100,100,1.63,CR,1655781881
545,Verāval,20.9000,70.3667,85.53,75,42,16.04,IN,1655781881
546,Novyy Urgal,51.0736,132.5642,57.83,97,100,4.79,RU,1655781882


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities_df2 = cities_df[(cities_df['Max Temp'] < 80)]                        # matching the parameters specified in the Readme for potential Hotel destinations
cities_df3 = cities_df2[(cities_df2['Max Temp'] > 70)]
cities_df4 = cities_df3[(cities_df3['Wind Speed'] < 10)]
hotel_df = cities_df4[(cities_df4['Cloudiness'] == 00)]
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,Fallon,39.4735,-118.7774,71.47,17,0,8.05,US,1655781497
36,Charter Township of Clinton,42.5870,-82.9199,77.90,78,0,0.00,US,1655780928
39,Mayor Pablo Lagerenza,-19.9309,-60.7718,72.07,53,0,7.09,PY,1655781509
68,Béja,36.7256,9.1817,71.26,56,0,3.47,TN,1655781530
78,Crotone,39.0851,17.1178,73.18,57,0,2.24,IT,1655781536
91,Bella Vista,36.4295,-94.2316,78.04,70,0,5.75,US,1655781546
110,Lompoc,34.6391,-120.4579,75.34,71,0,8.05,US,1655781560
144,Plainview,40.7765,-73.4673,71.46,68,0,3.44,US,1655781585
146,Nardaran,40.5561,50.0056,74.84,64,0,6.91,AZ,1655781586
160,Province of Trapani,37.8333,12.6667,73.27,77,0,4.61,IT,1655781596


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df

c:\Users\Brendan\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
24,Fallon,39.4735,-118.7774,71.47,17,0,8.05,US,1655781497,
36,Charter Township of Clinton,42.5870,-82.9199,77.90,78,0,0.00,US,1655780928,
39,Mayor Pablo Lagerenza,-19.9309,-60.7718,72.07,53,0,7.09,PY,1655781509,
68,Béja,36.7256,9.1817,71.26,56,0,3.47,TN,1655781530,
78,Crotone,39.0851,17.1178,73.18,57,0,2.24,IT,1655781536,
91,Bella Vista,36.4295,-94.2316,78.04,70,0,5.75,US,1655781546,
110,Lompoc,34.6391,-120.4579,75.34,71,0,8.05,US,1655781560,
144,Plainview,40.7765,-73.4673,71.46,68,0,3.44,US,1655781585,
146,Nardaran,40.5561,50.0056,74.84,64,0,6.91,AZ,1655781586,
160,Province of Trapani,37.8333,12.6667,73.27,77,0,4.61,IT,1655781596,


In [7]:
target_search = "Hotel"
target_radius = 5000
target_type = "Hotel"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
        lat = row["Lat"]
        lng = row["Lng"]
        params["location"] = f"{lat},{lng}"
        response = requests.get(base_url, params=params).json()
        results = response['results']

        try:
            hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
        except (KeyError, IndexError):
            print("Missing field/result... skipping.")
        
            print("------------")
   #pprint(results)hotel_df

c:\Users\Brendan\anaconda3\envs\bootcamp\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------


In [8]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
24,Fallon,39.4735,-118.7774,71.47,17,0,8.05,US,1655781497,Best Western Fallon Inn & Suites
36,Charter Township of Clinton,42.5870,-82.9199,77.90,78,0,0.00,US,1655780928,Hampton Inn Detroit/Roseville
39,Mayor Pablo Lagerenza,-19.9309,-60.7718,72.07,53,0,7.09,PY,1655781509,
68,Béja,36.7256,9.1817,71.26,56,0,3.47,TN,1655781530,RamSam Hotel
78,Crotone,39.0851,17.1178,73.18,57,0,2.24,IT,1655781536,Hotel Ristorante Lido degli Scogli
91,Bella Vista,36.4295,-94.2316,78.04,70,0,5.75,US,1655781546,Comfort Inn Bentonville
110,Lompoc,34.6391,-120.4579,75.34,71,0,8.05,US,1655781560,Hilton Garden Inn Lompoc
144,Plainview,40.7765,-73.4673,71.46,68,0,3.44,US,1655781585,Hilton Long Island/Huntington
146,Nardaran,40.5561,50.0056,74.84,64,0,6.91,AZ,1655781586,Bilgah Beach Hotel
160,Province of Trapani,37.8333,12.6667,73.27,77,0,4.61,IT,1655781596,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))